In [2]:
from web3 import Web3
import time

# Configuração da conexão
INFURA_URL = "https://mainnet.infura.io/v3/079d35a3c196469191c9accc1d68d25b"
web3 = Web3(Web3.HTTPProvider(INFURA_URL))




In [4]:
# UNISWAP_FACTORY_ADDRESS = web3.to_checksum_address("0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f")
# UNISWAP_FACTORY_ABI = """[
#     {"constant":true,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"view","type":"function"}
# ]"""

# factory_contract = web3.eth.contract(address=UNISWAP_FACTORY_ADDRESS, abi=UNISWAP_FACTORY_ABI)

# # Verifique o endereço do pool para dois tokens (USDC e WETH, por exemplo)
# TOKEN_A = web3.to_checksum_address("0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eb48")  # USDC
# TOKEN_B = web3.to_checksum_address("0xC02aaa39b223FE8D0a0e5C4F27eAD9083C756Cc2")  # WETH
# pool_address = factory_contract.functions.getPair(TOKEN_A, TOKEN_B).call()

# print(f"Endereço do Pool: {pool_address}")


# # Configuração do contrato
# pool_contract = web3.eth.contract(address=pool_address, abi=POOL_ABI)

# def get_pool_prices():
#     # Obtém as reservas do pool
#     reserves = pool_contract.functions.getReserves().call()
#     reserve0, reserve1 = reserves[0], reserves[1]
    
#     # Calcular preço (considerando tokens na ordem da pool)
#     price_token0_in_token1 = reserve1 / reserve0
#     price_token1_in_token0 = reserve0 / reserve1
    
#     return price_token0_in_token1, price_token1_in_token0


# # Monitorar continuamente
# try:
#     print("Monitorando o pool...")
#     while True:
#         price_token0, price_token1 = get_pool_prices()
#         print(f"Preço Token0/Token1: {price_token0:.12f}, Preço Token1/Token0: {price_token1:.12f}")
#         time.sleep(5)  # Atualizar a cada 5 segundos
# except KeyboardInterrupt:
#     print("Monitoramento interrompido.")
    


In [5]:

# Endereço e ABI da Uniswap Factory
UNISWAP_FACTORY_ADDRESS = web3.to_checksum_address("0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f")
UNISWAP_FACTORY_ABI = """[
    {"constant":true,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"view","type":"function"}
]"""
POOL_ABI = """[
    {"constant":true,"inputs":[],"name":"getReserves","outputs":[
        {"internalType":"uint112","name":"_reserve0","type":"uint112"},
        {"internalType":"uint112","name":"_reserve1","type":"uint112"},
        {"internalType":"uint32","name":"_blockTimestampLast","type":"uint32"}
    ],"payable":false,"stateMutability":"view","type":"function"}
]"""

ERC20_ABI = """[
    {"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"}
]"""

def get_token_decimals(token_address):
    token_contract = web3.eth.contract(address=web3.to_checksum_address(token_address), abi=ERC20_ABI)
    return token_contract.functions.decimals().call()
    
# Inicializa o contrato da Uniswap Factory
factory_contract = web3.eth.contract(address=UNISWAP_FACTORY_ADDRESS, abi=UNISWAP_FACTORY_ABI)

# Lista de pares de tokens para monitorar
TOKEN_PAIRS = [
    {"name": "USDC/WETH", "tokenA": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "tokenB": "0xC02aaa39b223FE8D0a0e5C4F27eAD9083C756Cc2"},  # USDC/WETH
    {"name": "DAI/WETH", "tokenA": "0x6b175474e89094c44da98b954eedeac495271d0f", "tokenB": "0xC02aaa39b223FE8D0a0e5C4F27eAD9083C756Cc2"}   # DAI/WETH
]

# Obter as decimais dos tokens
for pair in TOKEN_PAIRS:
    pair["decimals_tokenA"] = get_token_decimals(pair["tokenA"])
    pair["decimals_tokenB"] = get_token_decimals(pair["tokenB"])
    
# Função para obter o endereço do pool para um par de tokens
def get_pool_address(tokenA, tokenB):
    return factory_contract.functions.getPair(
        web3.to_checksum_address(tokenA),
        web3.to_checksum_address(tokenB)
    ).call()


# Função para obter preços ajustados
def get_pool_prices(pool_contract, decimals_token0, decimals_token1):
    reserves = pool_contract.functions.getReserves().call()
    reserve0, reserve1 = reserves[0], reserves[1]
    
    # Ajustar reservas para a mesma escala
    adjusted_reserve0 = reserve0 / (10 ** decimals_token0)
    adjusted_reserve1 = reserve1 / (10 ** decimals_token1)
    
    # Calcular preços ajustados
    price_token0_in_token1 = adjusted_reserve1 / adjusted_reserve0
    price_token1_in_token0 = adjusted_reserve0 / adjusted_reserve1
    
    return price_token0_in_token1, price_token1_in_token0

# Inicializa os contratos dos pools
pools = []
for pair in TOKEN_PAIRS:
    pool_address = get_pool_address(pair["tokenA"], pair["tokenB"])
    if pool_address == "0x0000000000000000000000000000000000000000":
        print(f"Pool não encontrado para o par {pair['name']}")
        continue
    pool_contract = web3.eth.contract(address=pool_address, abi=POOL_ABI)
    pools.append({"name": pair["name"], "contract": pool_contract})


def calculate_spread(price_a, price_b):
    """
    Calcula o spread percentual entre dois preços.

    Args:
        price_a (float): Preço do Token0/Token1 no primeiro pool.
        price_b (float): Preço do Token0/Token1 no segundo pool.

    Returns:
        float: Spread percentual (valor absoluto).
    """
    spread = abs(price_a - price_b) / min(price_a, price_b) * 100
    return spread

try:
    print("Monitorando múltiplos pools com escalas ajustadas e calculando spreads...")
    while True:
        # Armazenar preços para comparação
        prices = {}
        for pool, pair in zip(pools, TOKEN_PAIRS):
            price_token0, price_token1 = get_pool_prices(
                pool["contract"], pair["decimals_tokenA"], pair["decimals_tokenB"]
            )
            prices[pair["name"]] = {"price_token0": price_token0, "price_token1": price_token1}
            print(f"{pair['name']}: Preço Token0/Token1: {price_token0:.12f}, Preço Token1/Token0: {price_token1:.12f}")

        # Comparar preços entre pools
        pool_names = list(prices.keys())
        print("\nComparando spreads entre pools...")
        for i in range(len(pool_names) - 1):
            pool_a = pool_names[i]
            pool_b = pool_names[i + 1]
            price_a = prices[pool_a]["price_token0"]
            price_b = prices[pool_b]["price_token0"]
            spread = calculate_spread(price_a, price_b)
            print(f"Spread entre {pool_a} e {pool_b}: {spread:.6f}%")
        print("-" * 30)
        time.sleep(5)
except KeyboardInterrupt:
    print("Monitoramento interrompido.")


Monitorando múltiplos pools com escalas ajustadas e calculando spreads...
USDC/WETH: Preço Token0/Token1: 0.000255483015, Preço Token1/Token0: 3914.154520756662
DAI/WETH: Preço Token0/Token1: 0.000255852550, Preço Token1/Token0: 3908.501201612174

Comparando spreads entre pools...
Spread entre USDC/WETH e DAI/WETH: 0.144642%
------------------------------
USDC/WETH: Preço Token0/Token1: 0.000255483015, Preço Token1/Token0: 3914.154520756662
DAI/WETH: Preço Token0/Token1: 0.000255852550, Preço Token1/Token0: 3908.501201612174

Comparando spreads entre pools...
Spread entre USDC/WETH e DAI/WETH: 0.144642%
------------------------------
USDC/WETH: Preço Token0/Token1: 0.000255286763, Preço Token1/Token0: 3917.163534492316
DAI/WETH: Preço Token0/Token1: 0.000255852550, Preço Token1/Token0: 3908.501201612174

Comparando spreads entre pools...
Spread entre USDC/WETH e DAI/WETH: 0.221628%
------------------------------
USDC/WETH: Preço Token0/Token1: 0.000255286763, Preço Token1/Token0: 3917.

In [6]:
def check_liquidity_and_slippage(pool_contract, decimals_token0, decimals_token1, volume, max_slippage=0.01):
    """
    Verifica a liquidez do pool e calcula o impacto de slippage.

    Args:
        pool_contract: Contrato do pool.
        decimals_token0: Decimais do Token0.
        decimals_token1: Decimais do Token1.
        volume: Volume a ser negociado (em unidades de Token0).
        max_slippage: Slippage máxima permitida (ex.: 0.01 para 1%).

    Returns:
        dict: Resultado com slippage, status e liquidez disponível.
    """
    reserves = pool_contract.functions.getReserves().call()
    reserve0, reserve1 = reserves[0], reserves[1]

    # Ajustar para a escala da maior quantidade de decimais
    common_scale = max(decimals_token0, decimals_token1)
    adjusted_reserve0 = reserve0 / (10 ** (common_scale - decimals_token0))
    adjusted_reserve1 = reserve1 / (10 ** (common_scale - decimals_token1))

    # Simular nova reserva após a negociação
    new_reserve0 = adjusted_reserve0 + volume
    new_reserve1 = adjusted_reserve1 * adjusted_reserve0 / new_reserve0

    # Calcular preço inicial e final
    price_initial = adjusted_reserve1 / adjusted_reserve0
    price_final = new_reserve1 / new_reserve0

    # Calcular slippage
    slippage = abs(price_final - price_initial) / price_initial

    # Verificar se a slippage está dentro do limite
    status = "Aceitável" if slippage <= max_slippage else "Excessivo"

    return {
        "slippage": slippage,
        "status": status,
        "liquidity_token0": adjusted_reserve0,
        "liquidity_token1": adjusted_reserve1,
    }


In [7]:
from web3 import Web3
import time

# Configuração das DEXs e Factories
DEX_FACTORIES = {
    "Uniswap": {
        "factory_address": "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f",
        "rpc_url": "https://mainnet.infura.io/v3/079d35a3c196469191c9accc1d68d25b",
    },
    "SushiSwap": {
        "factory_address": "0xC0AEe478e3658e2610c5F7A4A2E1777Ce9e4f2Ac",
        "rpc_url": "https://mainnet.infura.io/v3/079d35a3c196469191c9accc1d68d25b",
    },
}

# ABI para contratos
UNISWAP_FACTORY_ABI = """[
    {"constant":true,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"view","type":"function"}
]"""

POOL_ABI = """[
    {"constant":true,"inputs":[],"name":"getReserves","outputs":[
        {"internalType":"uint112","name":"_reserve0","type":"uint112"},
        {"internalType":"uint112","name":"_reserve1","type":"uint112"},
        {"internalType":"uint32","name":"_blockTimestampLast","type":"uint32"}
    ],"payable":false,"stateMutability":"view","type":"function"}
]"""

ERC20_ABI = """[
    {"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"}
]"""

# Tokens a serem monitorados
TOKEN_PAIRS = [
    {"name": "USDC/USDT", "tokenA": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "tokenB": "0xdac17f958d2ee523a2206206994597c13d831ec7"},
    {"name": "USDC/DAI", "tokenA": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "tokenB": "0x6b175474e89094c44da98b954eedeac495271d0f"},
    {"name": "USDT/DAI", "tokenA": "0xdac17f958d2ee523a2206206994597c13d831ec7", "tokenB": "0x6b175474e89094c44da98b954eedeac495271d0f"},
]

# Inicializa conexões com Web3
web3_connections = {
    dex_name: Web3(Web3.HTTPProvider(dex_data["rpc_url"])) for dex_name, dex_data in DEX_FACTORIES.items()
}

# Função para obter as decimais de um token
def get_token_decimals(web3, token_address):
    token_contract = web3.eth.contract(address=web3.to_checksum_address(token_address), abi=ERC20_ABI)
    return token_contract.functions.decimals().call()

# Função para obter o endereço do pool
def get_pool_address(web3, factory_address, tokenA, tokenB):
    factory_contract = web3.eth.contract(address=web3.to_checksum_address(factory_address), abi=UNISWAP_FACTORY_ABI)
    return factory_contract.functions.getPair(
        web3.to_checksum_address(tokenA),
        web3.to_checksum_address(tokenB)
    ).call()

def get_pool_prices(pool_contract, decimals_token0, decimals_token1):
    reserves = pool_contract.functions.getReserves().call()
    reserve0, reserve1 = reserves[0], reserves[1]

    # Ajustar para a escala da maior quantidade de decimais
    common_scale = max(decimals_token0, decimals_token1)
    adjusted_reserve0 = reserve0 / (10 ** (common_scale - decimals_token0))
    adjusted_reserve1 = reserve1 / (10 ** (common_scale - decimals_token1))

    # Garantir que os valores ajustados não sejam zero
    if adjusted_reserve0 > 0 and adjusted_reserve1 > 0:
        # Calcular preços ajustados
        price_token0_in_token1 = adjusted_reserve1 / adjusted_reserve0
        price_token1_in_token0 = adjusted_reserve0 / adjusted_reserve1
    else:
        print("Aviso: Reservas ajustadas inválidas.")
        return 0, 0

    return price_token0_in_token1, price_token1_in_token0

# Função para calcular o spread
def calculate_spread(price_a, price_b):
    if price_a == 0 or price_b == 0:
        return 0
    spread = abs(price_a - price_b) / min(price_a, price_b) * 100
    return spread

# Função para calcular taxas e lucro
def calculate_fees_and_profit(spread, trade_volume, gas_price_gwei, gas_used, dex_fee=0.003, flash_loan_fee=0.0009):
    # Taxa de Gas
    gas_cost_eth = gas_price_gwei * gas_used * 1e-9  # Converte Gwei para ETH

    # Taxa de Swap
    swap_cost = trade_volume * dex_fee * 2  # 2 swaps

    # Taxa de Flash Loan
    flash_loan_cost = trade_volume * flash_loan_fee

    # Lucro Bruto
    gross_profit = trade_volume * spread / 100

    # Taxas Totais
    total_fees = gas_cost_eth + swap_cost + flash_loan_cost

    # Lucro Líquido
    net_profit = gross_profit - total_fees

    return {
        "gas_cost_eth": gas_cost_eth,
        "swap_cost": swap_cost,
        "flash_loan_cost": flash_loan_cost,
        "gross_profit": gross_profit,
        "total_fees": total_fees,
        "net_profit": net_profit
    }

# Inicializar pools
def initialize_pools():
    pools = []
    for pair in TOKEN_PAIRS:
        pair["decimals_tokenA"] = get_token_decimals(web3_connections["Uniswap"], pair["tokenA"])
        pair["decimals_tokenB"] = get_token_decimals(web3_connections["Uniswap"], pair["tokenB"])

        for dex_name, dex_data in DEX_FACTORIES.items():
            pool_address = get_pool_address(
                web3_connections[dex_name],
                dex_data["factory_address"],
                pair["tokenA"],
                pair["tokenB"]
            )

            if pool_address == "0x0000000000000000000000000000000000000000":
                print(f"Pool não encontrado para {pair['name']} na {dex_name}")
                continue

            pool_contract = web3.eth.contract(address=pool_address, abi=POOL_ABI)
            pools.append({"name": pair["name"], "dex": dex_name, "contract": pool_contract, "decimals": (pair["decimals_tokenA"], pair["decimals_tokenB"]), "address": pool_address})
    return pools

# Monitorar pools
def monitor_pools(pools, trade_volume=1000000, gas_price_gwei=30, gas_used=300000):
    try:
        print("Monitorando múltiplos pools...")
        while True:
            prices = {}
            for pool in pools:
                result = check_liquidity_and_slippage(
                    pool["contract"],
                    pool["decimals"][0],
                    pool["decimals"][1],
                    volume=trade_volume,
                    max_slippage=0.01,  # 1% de slippage máximo
                )
            
                print(f"Pool: {pool['name']} ({pool['dex']})")
                print(f"Slippage: {result['slippage']:.6f} ({result['status']})")
                print(f"Liquidez: Token0 = {result['liquidity_token0']:.2f}, Token1 = {result['liquidity_token1']:.2f}")
            
                if result["status"] == "Excessivo":
                    print(f"Volume {trade_volume} excede a liquidez segura para este pool.")
                else:
                    print("Liquidez adequada para arbitragem.")
                    
                price_token0, price_token1 = get_pool_prices(
                    pool["contract"],
                    pool["decimals"][0],
                    pool["decimals"][1]
                )
                pool_key = f"{pool['name']} ({pool['dex']})"
                prices[pool_key] = price_token0
                print(f"{pool_key}: Preço Token0/Token1: {price_token0:.12f}, Preço Token1/Token0: {price_token1:.12f}")

            print("\nComparando spreads entre pools...")
            pool_keys = list(prices.keys())
            for i in range(len(pool_keys) - 1):
                for j in range(i + 1, len(pool_keys)):
                    spread = calculate_spread(prices[pool_keys[i]], prices[pool_keys[j]])
                    if spread > 0:
                        fees_and_profit = calculate_fees_and_profit(
                            spread, trade_volume, gas_price_gwei, gas_used
                        )
                        print(f"Spread entre {pool_keys[i]} e {pool_keys[j]}: {spread:.6f}%")
                        print(f"Lucro Bruto: {fees_and_profit['gross_profit']:.2f} USD")
                        print(f"Taxas Totais: {fees_and_profit['total_fees']:.2f} USD")
                        print(f"Lucro Líquido: {fees_and_profit['net_profit']:.2f} USD")

            print("-" * 30)
            time.sleep(5)
    except KeyboardInterrupt:
        print("Monitoramento interrompido.")




In [8]:
# Inicializa os pools e inicia o monitoramento
pools = initialize_pools()
monitor_pools(pools)

Monitorando múltiplos pools...
Pool: USDC/USDT (Uniswap)
Slippage: 0.000001 (Aceitável)
Liquidez: Token0 = 2350806890988.00, Token1 = 2352355350871.00
Liquidez adequada para arbitragem.
USDC/USDT (Uniswap): Preço Token0/Token1: 1.000658692932, Preço Token1/Token0: 0.999341740659
Pool: USDC/USDT (SushiSwap)
Slippage: 0.002265 (Aceitável)
Liquidez: Token0 = 881427020.00, Token1 = 876692492.00
Liquidez adequada para arbitragem.
USDC/USDT (SushiSwap): Preço Token0/Token1: 0.994628564938, Preço Token1/Token0: 1.005400443192
Pool: USDC/DAI (Uniswap)
Slippage: 0.000003 (Aceitável)
Liquidez: Token0 = 736388074558.91, Token1 = 736251725032.00
Liquidez adequada para arbitragem.
USDC/DAI (Uniswap): Preço Token0/Token1: 0.999814840121, Preço Token1/Token0: 1.000185194170
Pool: USDC/DAI (SushiSwap)
Slippage: 0.000160 (Aceitável)
Liquidez: Token0 = 12460598813.48, Token1 = 12476695464.00
Liquidez adequada para arbitragem.
USDC/DAI (SushiSwap): Preço Token0/Token1: 1.001291803930, Preço Token1/Token0